# IMPORT

In [1]:
from __future__ import annotations
from qiskit import QuantumCircuit
from transpiler.qasm2_reader import load_qasm, from_qasm2_to_braket
from bqskit.ext import bqskit_to_qiskit,qiskit_to_bqskit
from qiskit_aer import AerSimulator
from qiskit import transpile
import pennylane as qml
import numpy as np
from braket.devices import LocalSimulator
from braket.circuits import Circuit as braket_Circuit
from transpiler.transpile import transpile_circuit as bqskit_transpiler
from transpiler.transpile import get_noisy_counts
from qiskit.circuit.random import random_circuit

In [11]:
# Create a 4-qubit quantum circuit
# Generate a random 4-qubit circuit with a depth of 5
qc = random_circuit(4,max_operands=2, depth=3, measure=False)

    
qc_qiskit = qc
qc_bqskit=qiskit_to_bqskit(qc)
print(qc)

# Initialize the latest AerSimulator
simulator = AerSimulator(method="statevector")

# Transpile the circuit for the simulator
qc_transpiled = transpile(qc_qiskit, simulator)

# Add instruction to save statevector
qc_transpiled.save_statevector()

# Run the circuit in statevector mode
result = simulator.run(qc_transpiled).result()

# Get the statevector
statevector = result.get_statevector()

# Print the statevector
print("Not transpiled result:",np.abs(np.array(statevector))**2)#check

                 ┌────┐                       ┌───┐                  »
q_0: ────────────┤ √X ├───────────────────────┤ H ├──────────────────»
             ┌───┴────┴───┐        ┌──────────┴───┴──────────┐┌─────┐»
q_1: ────────┤ Rx(1.7915) ├────────┤ U(1.2147,1.2307,4.1169) ├┤ Tdg ├»
             └───┬───┬────┘        └──────┬────────────┬─────┘└─────┘»
q_2: ────────────┤ S ├────────────────────┤ Rx(1.9395) ├─────────────»
     ┌───────────┴───┴────────────┐       ├───────────┬┘       ┌───┐ »
q_3: ┤ U3(5.0314,6.2677,0.014352) ├───────┤ P(5.2392) ├────────┤ Z ├─»
     └────────────────────────────┘       └───────────┘        └───┘ »
«                  
«q_0: ─■───────────
«      │           
«q_1: ─┼───────────
«      │ZZ(4.5347) 
«q_2: ─■───────────
«                  
«q_3: ─────────────
«                  
Not transpiled result: [0.0882542  0.0882542  0.01676195 0.01676195 0.18773259 0.18773259
 0.03565568 0.03565568 0.04611397 0.04611397 0.00875833 0.00875833
 0.09809272 0.09809272 0

In [12]:
# Step 3: Transpile the circuit to MS gates
qc_qiskit_transpiled = bqskit_transpiler(qc_qiskit,device="Aria",optimization_level=2,synthesis_epsilon=1e-4)
print(qc_qiskit_transpiled)

      ┌─────────────┐ ┌─────────────┐                 »
q_0: ─┤ Gpi(3.9897) ├─┤ Gpi(2.8905) ├─────────────────»
     ┌┴─────────────┴┐├─────────────┴┐┌─────────────┐ »
q_1: ┤ Gpi2(0.77019) ├┤ Gpi2(1.5922) ├┤ Gpi(3.7384) ├─»
     └┬─────────────┬┘├──────────────┤├─────────────┤ »
q_2: ─┤ Gpi(4.5998) ├─┤ Gpi2(2.9164) ├┤ Gpi(1.3404) ├─»
      ├─────────────┤ ├──────────────┤├─────────────┴┐»
q_3: ─┤ Gpi(3.5495) ├─┤ Gpi2(3.9718) ├┤ Gpi2(5.2236) ├»
      └─────────────┘ └──────────────┘└──────────────┘»
«     ┌───────────────────────────────────┐ ┌─────────────┐ ┌──────────────┐
«q_0: ┤0                                  ├─┤ Gpi(4.3986) ├─┤ Gpi2(3.1416) ├
«     │                                   │ └─────────────┘ └──────────────┘
«q_1: ┤  Partialms(0.94326,3.2747,1.3931) ├─────────────────────────────────
«     │                                   │┌───────────────┐┌──────────────┐
«q_2: ┤1                                  ├┤ Gpi(0.066548) ├┤ Gpi2(4.7124) ├
«     └───────────────────────────

# Convert Qiskit circuit to PennyLane


In [13]:
qml_circuit = qml.from_qiskit(qc_qiskit_transpiled)
print("Transpiled result",np.abs(qml.matrix(qml_circuit, wire_order=list(range(qc_qiskit.num_qubits)))().T[0])**2)#check
print("Not transpiled result:",np.abs(np.array(statevector))**2)#check

Transpiled result [0.08825422 0.04611398 0.18773258 0.09809271 0.01676195 0.00875834
 0.03565567 0.01863055 0.08825422 0.04611398 0.18773258 0.09809271
 0.01676195 0.00875834 0.03565567 0.01863055]
Not transpiled result: [0.0882542  0.0882542  0.01676195 0.01676195 0.18773259 0.18773259
 0.03565568 0.03565568 0.04611397 0.04611397 0.00875833 0.00875833
 0.09809272 0.09809272 0.01863056 0.01863056]


# NOISE

In [9]:
print("Noise 0     simulation result:", get_noisy_counts(qc_qiskit_transpiled,0.00,0.000,0.00))
print("Noisy     simulation result:", get_noisy_counts(qc_qiskit_transpiled,0.01,0.001,0.01))
print("Noiseless simulation result:",np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2)#check

Noise 0     simulation result: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


DeviceError: Shots must be a single non-negative integer or a sequence of non-negative integers.

# Vibratium

In [22]:
bracket_qc=from_qasm2_to_braket("circuit.qasm")


In [23]:
print(bracket_qc)

T  : │        0        │      1      │      2       │           3            │      4       │      5       │      6      │           7            │      8       │      9       │      10      │      11       │ 12  │
                        ┌───────────┐ ┌────────────┐ ┌──────────────────────┐ ┌────────────┐ ┌────────────┐ ┌───────────┐ ┌──────────────────────┐ ┌────────────┐ ┌────────────┐ ┌────────────┐                 ┌───┐ 
q0 : ───StartVerbatim───┤ GPi(3.93) ├─┤ GPi2(3.14) ├─┤ MS(1.16, 3.06, 1.32) ├─┤ GPi2(0.64) ├─┤ GPi2(6.20) ├─┤ GPi(3.12) ├─┤ MS(1.61, 0.10, 1.37) ├─┤ GPi2(1.19) ├─┤ GPi2(1.24) ├─┤ GPi2(5.58) ├───EndVerbatim───┤ M ├─
              ║         └───────────┘ └────────────┘ └──────────┬───────────┘ └────────────┘ └────────────┘ └───────────┘ └──────────┬───────────┘ └────────────┘ └────────────┘ └────────────┘        ║        └───┘ 
              ║         ┌───────────┐                ┌──────────┴───────────┐ ┌────────────┐                              ┌──────────┴──────

In [24]:
device = LocalSimulator() # Choose the local simulator
shots = 100000  # Number of times to execute the circuit

result = device.run(bracket_qc, shots=shots).result()
counts = result.measurement_probabilities  # Get the shot results

print(counts)  # Dictionary {bitstring: counts}
print("Noiseless simulation result:",np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2)#check

{'11': 0.35773, '01': 0.123, '10': 0.41512, '00': 0.10415}
Noiseless simulation result: [0.10533638 0.12272559 0.41539821 0.35653982]


# GATE braket TEST

In [17]:
print(braket_Circuit().gpi(0,1))

T  : │    0     │
      ┌────────┐ 
q0 : ─┤ GPi(1) ├─
      └────────┘ 
T  : │    0     │


In [18]:
braket_Circuit().gpi(0,0.445).to_unitary()

array([[0.        +0.j        , 0.90261067-0.43045788j],
       [0.90261067+0.43045788j, 0.        +0.j        ]])

In [19]:
phi=0.445
np.array([
            [0, np.exp(-1j * phi)],
            [np.exp(1j * phi), 0]
        ])

array([[0.        +0.j        , 0.90261067-0.43045788j],
       [0.90261067+0.43045788j, 0.        +0.j        ]])

In [20]:
braket_Circuit().gpi2(0,0.445).to_unitary()

array([[ 0.70710678+0.j        , -0.30437969-0.63824212j],
       [ 0.30437969-0.63824212j,  0.70710678+0.j        ]])

In [13]:
 # Extract the phase parameter
factor = 1 / np.sqrt(2)  # Normalization factor
exp_pos = -1j * np.exp(1j * phi)
exp_neg = -1j * np.exp(-1j * phi)

factor * np.array([
            [1, exp_neg],
            [exp_pos, 1]
        ])

array([[ 0.70710678+0.j        , -0.30437969-0.63824212j],
       [ 0.30437969-0.63824212j,  0.70710678+0.j        ]])

In [14]:
phi1=0.2455
phi2=0.4157
theta=0.11545
q1=0
q2=1
braket_Circuit().ms(q1, q2, phi1, phi2,theta).to_unitary()

array([[ 0.99833437+0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.03542718-0.0455345j ],
       [ 0.        +0.j        ,  0.99833437+0.j        ,
         0.009772  -0.05685934j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.009772  -0.05685934j,
         0.99833437+0.j        ,  0.        +0.j        ],
       [ 0.03542718-0.0455345j ,  0.        +0.j        ,
         0.        +0.j        ,  0.99833437+0.j        ]])

In [15]:
phi0=phi1
phi1=phi2

cos_theta = np.cos(theta/2)
sin_theta = np.sin(theta/2)

e_pos = -1j * np.exp(1j * (phi0 + phi1))
e_neg = -1j * np.exp(-1j * (phi0 + phi1))
e_diff_pos = -1j * np.exp(1j * (phi0 - phi1))
e_diff_neg = -1j * np.exp(-1j * (phi0 - phi1))

np.array([
    [cos_theta, 0, 0, e_neg * sin_theta],
    [0, cos_theta, e_diff_neg * sin_theta, 0],
    [0, e_diff_pos * sin_theta, cos_theta, 0],
    [e_pos * sin_theta, 0, 0, cos_theta]
])

array([[ 0.99833437+0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.03542718-0.0455345j ],
       [ 0.        +0.j        ,  0.99833437+0.j        ,
         0.009772  -0.05685934j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.009772  -0.05685934j,
         0.99833437+0.j        ,  0.        +0.j        ],
       [ 0.03542718-0.0455345j ,  0.        +0.j        ,
         0.        +0.j        ,  0.99833437+0.j        ]])

In [16]:
braket_Circuit().zz(0,1,np.pi).to_unitary()

array([[6.123234e-17-1.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 6.123234e-17+1.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 6.123234e-17+1.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        6.123234e-17-1.j]])